In [1]:
import numpy as np
import pandas as pd
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.layers import Input, Embedding, Bidirectional, LSTM, GlobalMaxPooling1D, Dense
# from keras import regularizers
# from keras.models import Model
# from keras.utils import to_categorical

with open('data/additional/preprocessed_data.json') as f:
    data = pd.read_json(f)

In [ ]:
headers = list(data.Header)
train = headers[:5000]
test = headers[-500:]
del data

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train)

In [3]:
MAX_LEN = max(len(line.split()) for line in train)
VOCAB_SIZE = len(tokenizer.word_index)+1
print("Vocab size:", VOCAB_SIZE)
print("Max lenght:", MAX_LEN)

Vocab size: 10120
Max lenght: 34


In [4]:
def encode_sequences(tokenizer, length, lines):
    X = tokenizer.texts_to_sequences(lines)
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

def encode_output(sequences, vocab_size):
    ylist = []
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = np.array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

In [5]:
from keras.models import Sequential
from keras.layers import RepeatVector, TimeDistributed
def define_model(vocab_size, timesteps, n_units):
    model = Sequential()
    model.add(Embedding(vocab_size, n_units, input_length=timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))
    return model

In [6]:
trainX = encode_sequences(tokenizer, MAX_LEN, train)
trainY = encode_output(trainX, VOCAB_SIZE)

testX = encode_sequences(tokenizer, MAX_LEN, test)
testY = encode_output(testX, VOCAB_SIZE)

model = define_model(VOCAB_SIZE, MAX_LEN, 128)
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(trainX, trainY, epochs=30, validation_data=(testX,testY))

Epoch 1/30
157/157 [==============================] - 69s 439ms/step - loss: 3.3589 - val_loss: 2.0643
Epoch 2/30
157/157 [==============================] - 68s 431ms/step - loss: 2.1422 - val_loss: 1.9383
Epoch 3/30
157/157 [==============================] - 67s 429ms/step - loss: 2.0087 - val_loss: 1.8829
Epoch 4/30
157/157 [==============================] - 68s 435ms/step - loss: 1.9626 - val_loss: 1.8718
Epoch 5/30
157/157 [==============================] - 68s 433ms/step - loss: 1.9012 - val_loss: 1.8512
Epoch 6/30
157/157 [==============================] - 68s 434ms/step - loss: 1.8542 - val_loss: 1.8335
Epoch 7/30
157/157 [==============================] - 67s 429ms/step - loss: 1.8278 - val_loss: 1.8276
Epoch 8/30
157/157 [==============================] - 70s 443ms/step - loss: 1.7998 - val_loss: 1.8248
Epoch 9/30
157/157 [==============================] - 68s 430ms/step - loss: 1.7752 - val_loss: 1.8323
Epoch 10/30
157/157 [==============================] - 75s 477ms/step - l

In [10]:
def word_for_id(integer, tokenizer):
    for word, i in tokenizer.word_index.items():
        if i == integer:
            return word
    return None

def predict_sequence(model, tokenizer, source, i):
    prediction = model.predict(source, verbose=0)[i]
    integers = [np.argmax(vector) for vector in prediction]
    target = []
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

In [35]:
#predict_sequence(model, tokenizer, trainX, 1)
integers = None
testX[0], testY[0]

(array([  32, 1409, 2042,   67,   43,   41,   32, 8895,    9,   14, 6450,
          65,  247,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]], dtype=float32))

In [46]:
preds = model.predict(testX)
integers = [np.argmax(v) for v in preds[0]]
print(integers)
[word_for_id(i, tokenizer) for i in integers]

[38, 12, 5, 5, 9, 9, 14, 8, 8, 8, 12, 12, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


['ny',
 'og',
 'i',
 'i',
 'til',
 'til',
 'at',
 'på',
 'på',
 'på',
 'og',
 'og',
 '3f',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [45]:
integers = [np.argmax(v) for v in testY[0]]
print(integers)
[word_for_id(i, tokenizer) for i in integers]

[32, 1409, 2042, 67, 43, 41, 32, 8895, 9, 14, 6450, 65, 247, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


['de',
 'kalder',
 'ham',
 'se',
 'der',
 'får',
 'de',
 'professionelle',
 'til',
 'at',
 'måbe',
 'tv',
 '2',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [19]:
integers

[2321,
 2322,
 3810,
 76,
 19,
 708,
 3811,
 611,
 364,
 19,
 3812,
 1,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [37]:
with open('headers.txt', 'w', encoding='utf-8') as f:
    strheaders = [x.replace('\n','') for x in headers if len(x)>10]
    strheaders = [x.replace('\t','') for x in strheaders]
    strheaders = '\n'.join(strheaders)
    f.write(strheaders)

In [17]:
strheaders = [x.replace('\n','') for x in headers if len(x)>10]
print(len(strheaders))
strheaders = '\n'.join(strheaders)

strheaders = strheaders.split('\n')
print(len(strheaders))
np.argmin([len(x) for x in strheaders])

308119
308119


38461

In [21]:
with open('headers.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(strheaders))

In [87]:
np.argmin([len(x) for x in splt])

38461

In [15]:
[x for x in strheaders if '\\' in x]

[]

In [88]:
splt[38461]

'Lukket fest'

In [38]:
with open('headers.txt', 'r', encoding='utf-8') as f:
    h = f.read()

In [39]:
h = headers.split('\n')

In [40]:
np.argmin([len(x) for x in h])

0

In [43]:
headers

IndexError: list index out of range

# More testing

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
import os

def load_data():
    path = 'data/additional/scraped'
    files = os.listdir(path)
    dataframes = []
    for file in tqdm(files):
        dataframes.append(pd.read_json(path+'/'+file))

    dataframe = pd.concat(dataframes)
    del dataframe['level_0']
    del dataframe['index']

    return dataframe

data = load_data()
domains = data.Domain.value_counts()
domains1000 = domains[domains>=1000]
domains1000_list = list(domains1000.index)

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:08<00:00,  3.47it/s]


In [16]:
data[data.Domain=='sn.dk']

,Id,Domain,Body,Header,PublicationDate,Uri,Byline,TextHash
12489,828470,sn.dk,"Sorø - 19. juni 2013 kl. 12:01 Af Rasmus Giese JakobsenFrederiksberg: Et legehus på legepladsen ved Frederiksberg Skole udbrændte natten til onsdag. Branden blev opdaget kl. 02.07, men på det tidspunkt stod flammerne allerede så højt, at Sorø Brandvæsen måtte opgive at redde legehuset. - Det brændte voldsomt, da vi nåede frem, men heldigvis stod det så langt væk fra de øvrige bygninger på skol...",Påsat brand i legehus - Sorø,2019-06-13T00:00:00,https://sn.dk/Soroe/Paasat-brand-i-legehus/artikel/280608,Rasmus Giese Jakobsen,2146540339
12490,828466,sn.dk,"Stjal varebil fuld af pakker Opdateret 10. juli 2009 kl. 15:34 Ringsted - 09. juli 2009 kl. 10:38 Af Rasmus Giese JakobsenRingsted: En kun 19-årig chauffør fik onsdag morgen stjålet den varebil, han kørte for pakkefirmaet GLS. Undervejs på ruten standsede den 19-årige den store varebile på Allikevej i Havbyrd ved Ringsted for at checke pakkerne i varerummet. Da chaufføren stod og så ind i vare...",Stjal varebil fuld af pakker - Ringsted,2009-07-09T00:00:00,https://sn.dk/Ringsted/Stjal-varebil-fuld-af-pakker/artikel/5724,Rasmus Giese Jakobsen,1336834975
12491,828458,sn.dk,"Opdateret 16. juni 2009 kl. 13:43 Ringsted - 15. juni 2009 kl. 12:38 Af Rasmus Giese JakobsenRingsted: Fredag eftermiddag eller nat forsvandt der en turkis lastbil fra Tømrergården i Ringsted. Lastbilen, der har registrering{AVIS}ummeret RU 91 240, blev brugt til at køre stjålne gulve og døre væk med. Vidner så en polsk indregistreret Passat på stedet, som det lykkedes politiet at opspore ejer...",Stjal gulv og døre - Ringsted,2015-06-09T00:00:00,https://sn.dk/Ringsted/Stjal-gulv-og-doere/artikel/1371,Rasmus Giese Jakobsen,169717534
12492,828452,sn.dk,"144 i timen på landevejen Sorø - 19. juli 2013 kl. 13:40 Af Rasmus Giese JakobsenSORØ: En bilist havde mere end almindeligt travlt i går på Sorøvej ved Dianalund. Politiet havde sin automatiske trafikkontrol stillet op og kameraet fangede en, der kørte hele 144 km/t på strækningen, hvor man kun må køre 80 km/t. Det koster en ubetinget frakendelse af kørekortet. Kontrollen stod på Sorøvej melle...",144 i timen på landevejen - Sorø,2019-07-13T00:00:00,https://sn.dk/Soroe/144-i-timen-paa-landevejen/artikel/342057,Rasmus Giese Jakobsen,580636992
12493,828449,sn.dk,"Teatertrup har styr på hvert minut Sorø - 14. november 2012 kl. 11:45 Af Rasmus Giese Jakobsen SORØ: Hvert minut og hvert et trin er planlagt. Det kræver en stramt koordineret plan at invitere 80 mennesker til et juleshow, når skuespillerne både skal stå på scenen og servere karrysild og sylte til publikum. Det ved medlemmerne af teatertruppen Bravour alt om. Den 12. til 15. december opfører d...",Teatertrup har styr på hvert minut - Sorø,2014-11-12T00:00:00,https://sn.dk/Soroe/Teatertrup-har-styr-paa-hvert-minut/artikel/237290,Rasmus Giese Jakobsen,126046121
...,...,...,...,...,...,...,...,...
284688,610440,sn.dk,"Hun er kendt for en lang række realityprogrammer som 'Divaer i junglen' og 'Paradise Hotel', men for 24-årige Nikita Klæstrup er det nu slut med at stå foran kameraet. Det fortalte hun, da Ekstra Bladet mødte hende til premieren på femte sæson af tv-serien C More-serien 'Peaky Blinders' i Empire Bio torsdag aften. - I kommer ikke til at se mig på tv, nej. Jeg kan meget bedre lide at være bag k...",Dropper tv: - Det er slut,2013-02-20T00:00:00,https://sn.dk/Taastrup/Elever-i-kaempe-projekt-Saadan-ser-vores-by-ud-om-40-aar/artikel/913666,Kasper Ellesøe,1909048407
284689,610438,sn.dk,"Det blev en dyr fornøjelse for en yngre mand fra Randers, da han trykkede speederen i bund og tonsede ud ad motorvejen ved Haderslev med 217 kilometer i timen. På vejen overhalede han nemlig en af politiets civile færdselspatruljer. Patruljen fik stoppet manden blot for at finde ud af, at han i forvejen havde et kørselsforbud. Det endte med at koste manden en bøde på 15.000 kroner samt en frak...",Mand får stor bøde: Kørte forbi politiet 

In [29]:
data[data['Domain'].isin(domains1000_list)]['Domain'].value_counts()

politiken.dk           64772
ekstrabladet.dk        63545
computerworld.dk       21427
jv.dk                  11399
berlingske.dk          11148
jyllands-posten.dk      9875
bold.dk                 7813
dr.dk                   7485
bt.dk                   6760
stiften.dk              6137
soundvenue.com          5971
altinget.dk             5476
sn.dk                   5400
ing.dk                  5250
gaffa.dk                5090
tv2.dk                  4905
finans.dk               4285
version2.dk             3741
journalisten.dk         3165
information.dk          2947
fyens.dk                2785
nordjyske.dk            1823
ekkofilm.dk             1660
seoghoer.dk             1575
tv2east.dk              1552
billedbladet.dk         1501
finanswatch.dk          1456
fagbladet3f.dk          1419
ejendomswatch.dk        1347
borsen.dk               1339
motormagasinet.dk       1328
helsingordagblad.dk     1240
fodevarewatch.dk        1138
jiyan.dk                1065
pov.internatio

In [44]:
data[data['Domain']=='ing.dk'].Body.str.contains('{AVIS}')

9206      True
9207      True
9208     False
9209      True
9210      True
         ...  
53346    False
53347     True
53348     True
53349     True
58106     True
Name: Body, Length: 5250, dtype: bool

In [45]:
data.iloc[9206].Body

"0 Fødevareminister Mette Gjerskov (S) forlænger nu fristen for at søge om ændr{AVIS}er i randzonerne fra 1. oktober til 1. november 2012. Forlængelsen sker efter ønske fra brancheorganisationen Landbrug og Fødevarer. »Randzonerne træder i kraft på lørdag. For mig er det vigtigt, at udruln{AVIS}en sker i god ro og orden. Det vil jeg gøre mit til,« siger Mette Gjerskov til EPN.dk og fortsætter: »Derfor har jeg valgt at imødekomme Landbrug og Fødevarers ønske om at give de landmænd, der bliver hårdt ramt, ekstra tid til at søge om dispensation.« Randzonerne er de dyrkn{AVIS}sfrie bræmmer på 10 meter langs vandløb, søer og kystnære farvande, der skal gøre Danmark i stand til at opfylde EU's krav til vandkvalitet. Loven, der træder i kraft i morgen, forbyder gødn{AVIS} og sprøjtegifte i randzonerne. Læs mere på EPN.dkmere om nyhedsbrevene her . {AVIS}eniøren daglige nyheder"

In [5]:
import os
from tqdm.notebook import tqdm

path = 'data/subset'
files = os.listdir(path)
dataframes = []
for file in tqdm(files):
    dataframes.append(pd.read_json(path+'/'+file))

data = pd.concat(dataframes)
del data['level_0']
del data['index']
pd.set_option('display.max_colwidth', 400)

  0%|          | 0/28 [00:00<?, ?it/s]

In [10]:
data[data.Header.str.contains('Ekstra Bladet')]

,Id,Domain,Body,Header,PublicationDate,Uri,Byline,TextHash
477,841843,ekstrabladet.dk,"Håber turen bliver aflyst De tre piger er nået dertil, at de faktisk håber, at skolen vil aflyse turen. - Jeg tror, vi mest håber, at turen bliver aflyst. For så er det besluttet, at vi ikke kan komme afsted, siger Natasja Christensen. - Men hvorfor bliver I så ikke bare hjemme? - Det er jo fordi, det er vores studietur. Man er bange for at gå glip af det sociale. - Sammenholdet bliver jo styr...",Skal på studietur til Rom: Vi håber turen bliver aflyst – Ekstra Bladet,2027-02-20T00:00:00,https://ekstrabladet.dk/nyheder/samfund/skal-paa-studietur-til-rom-vi-haaber-turen-bliver-aflyst/8022035,Emma Busk,667492848
478,841842,ekstrabladet.dk,"20. maj. 2020 kl. 20:52TV2-par: - Det var en kæmpe overraskelse Det var ægteparret Cassia og Charles, der løb med titlen som vindere af TV2-programmet 'Sommerdrømme' Det var to glade deltagere, der efter onsdagens finaleafsnit af 'Sommerdrømme' kunne fejre, at de nu er sommerhusejere. Foto: Jørn Deleuran/TV 2 Følger Film og tv Ups! Denne funktion kræver en gratis konto ... ... så vi kan gemme,...",TV2-par:Det var en kæmpe overraskelse – Ekstra Bladet,2020-05-20T00:00:00,https://ekstrabladet.dk/underholdning/filmogtv/tv2-par-det-var-en-kaempe-overraskelse/8127503,Emma Busk,662295962
479,841841,ekstrabladet.dk,"4. nov. 2019 kl. 13:33Opret konto Log ind Dyr navneændring hos Klima-Dan: Rykker ét ord Det har kostet over 140.000 kroner at flytte rundt på ordene i det tidligere Energi-, Forsynings- og Klimaministerium, så ordet 'klima' nu er rykket forrest Ud over at ændre navne og logoer vil Klima-Dan reducere Danmarks CO2-udledning med 70 procent. Men kan det overhovedet lade sig gøre? Det spurgte Ekstr...",Dyr navneændring hos Klima-Dan: Rykker ét ord – Ekstra Bladet,2004-11-19T00:00:00,https://ekstrabladet.dk/nyheder/politik/danskpolitik/dyr-navneaendring-hos-klima-dan-rykker-et-ord/7860364,Emma Busk,-1847341560
480,841840,ekstrabladet.dk,"Politiet i det sydlige Danmark har beslaglagt 31 kilo kokain i forbindelse med efterforskningen af en sag om narkosmugling. Det oplyser Syd- og Sønderjyllands Politi i en pressemeddelelse natten til tirsdag. En 34-årig udenlandsk mand er anholdt og varetægtsfængslet i sagen. - Det er sjældent, at vi kan beslaglægge så store mængder kokain på en gang. Der er tale om meget grov, organiseret krim...",Politiet beslaglægger 31 kilo kokain og anholder mistænkt – Ekstra Bladet,2003-03-20T00:00:00,https://ekstrabladet.dk/krimi/politiet-beslaglaegger-31-kilo-kokain-og-anholder-mistaenkt/8028749,Emma Busk,1621282620
481,841839,ekstrabladet.dk,"Det skriver Se og Hør , som henviser til salgsopstillingen . Ifølge Tingbogen har ægteparret ejet den 172 kvadratmeter store lejlighed siden sommeren 2017. Dengang måtte de slippe 6.750.000 kroner for den eksklusive bolig, som ligger et stenkast fra Nyhavn. Dermed kan de altså - såfremt lejligheden sælges til prisen - hive en fortjeneste hjem på 3.245.000 kroner. Tina Bilsbo skriver på Instagr...",Vil score millioner – Ekstra Bladet,2018-02-21T00:00:00,https://ekstrabladet.dk/underholdning/dkkendte/vil-score-millioner/8477214,Emma Busk,1886323114
...,...,...,...,...,...,...,...,...
4190,471947,ekstrabladet.dk,"Tre og fire stjerner er tilsyneladende ikke nok for, metal-drengene i Volbeat, der bestemt ikke er begejstrede over deres seneste anmeldelser i Ekstra Bladet\nLÆS OGSÅ:Fallulah scorede 100.000 kroner til P3 Guld Et af aftenens mere bizarre optrin til dette års P3 Guld-uddeling i DR's Koncerthus stod Volbeat-forsangeren Michael Poulsen for, da han overraskende valgte at bruge sin takketid efter...",Volbeat svinede Ekstra Bladet,2014-01-11T00:00:00,https://ekstrabladet.dk/musik/dkmusiknyt/article4120617.ece,Rune Melchior Sjørvad,0
4222,471342,ekstrabladet.dk,"Efter torsdagens kritiske Justin Bieber-anmeldelse har Ekstra Bladet været genstand for de såkaldte Beliebers' voldsomme raseri. Nu undskylder de krænkede fans truslerne\nANMELDELSEN, DE